# ANALYSIS OF BUSINESS CASE: WHICH NEIGHBORHOOD TO OPEN A COFFEE SHOP IN

## A client is looking to open a boutique coffee shop in Toronto.  The client is looking for a data based solution to find the best neighborhood to attempe this new venture.

### import the Pandas library

In [1]:
import pandas as pd

### read a list of postal codes for Toronto in a pd dataframe

In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df = pd.read_html(URL, converters={"Postal Code": str, "Borough": str, "Neighbourhood": str})

df = df[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### clean the data by removing any 'Not Assigned' values

In [6]:
not_assigned_index = df[df['Borough'] == 'Not assigned'].index
df.drop(not_assigned_index, inplace=True)

### display the cleaned data frame

In [8]:
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### now we will retrieve the coordinates for mapping and then merge all the data

In [10]:
df_coords = pd.read_csv("/Users/adamdecaria/Geospatial_Coordinates.csv")
df_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [16]:
mergedDf = df.merge(df_coords, on='Postal Code')
mergedDf

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [17]:
df_coords = pd.read_csv("/Users/adamdecaria/Geospatial_Coordinates.csv")
df_coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### install libraries necessary for the next steps

In [18]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### use geopy to get location data

In [19]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### create a map of Toronto using latitude and longitude values

In [20]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mergedDf['Latitude'], mergedDf['Longitude'], mergedDf['Borough'], mergedDf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

### let's explore two different neighborhoods the client is considering opening their business in

### first we will start with a location in downtown Toronto

In [95]:
downtown_data = mergedDf[mergedDf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [96]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent='downtown')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Downtown Toronto are 43.6534817, -79.3839347.


In [97]:
# create map of downtown Toronto - Church and Wellesley neighborhood

map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown) 

map_downtown

### now let's draw data from FourSquare

In [98]:
CLIENT_ID = '1KNHTHROYURQFE1LAE0IZYDP4VWEJXWOEQEMNEFSB2CXLPAY' # Foursquare ID
CLIENT_SECRET = '4ZS33NNGBV5JFCAIS3DJN3FKYAC2MCUNTRYZPI4XLATLZWBL' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [99]:
downtown_data.loc[18, 'Neighbourhood']

'Church and Wellesley'

In [100]:
neighbourhood_latitude = downtown_data.loc[18, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = downtown_data.loc[18, 'Longitude'] # neighborhood longitude value

neighbourhood_name = downtown_data.loc[18, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Church and Wellesley are 43.6658599, -79.38315990000001.


In [101]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [102]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [103]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Storm Crow Manor,Theme Restaurant,43.666840,-79.381593
1,DanceLifeX Centre,Dance Studio,43.666956,-79.385297
2,The Alley,Bubble Tea Shop,43.665922,-79.385567
3,Fabarnak,Restaurant,43.666377,-79.380964
4,Smith,Breakfast Spot,43.666927,-79.381421


In [104]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

75 venues were returned by Foursquare.


In [105]:
nearby_venues['categories']

0      Theme Restaurant
1          Dance Studio
2       Bubble Tea Shop
3            Restaurant
4        Breakfast Spot
            ...        
70    Food & Drink Shop
71                  Pub
72          Coffee Shop
73     Sushi Restaurant
74                Hotel
Name: categories, Length: 75, dtype: object

In [106]:
nearby_venues.groupby(by='categories').count()

,name,lat,lng
categories,,,
Afghan Restaurant,1,1,1
American Restaurant,1,1,1
Beer Bar,1,1,1
Bookstore,1,1,1
Breakfast Spot,1,1,1
Bubble Tea Shop,2,2,2
Burger Joint,1,1,1
Café,2,2,2
Caribbean Restaurant,1,1,1


### As per the above, there are 7 coffee shops already in this neighborhood; coffee shops significantly outnumber every other type of venue!  

### Let's look at the venues on a map of downtown to visualize it

In [110]:
# create map of downtown Toronto using latitude and longitude values
map_downtown_venues = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_venues)
    
map_downtown_venues

### Wow!  That's a pretty busy spot!  Not really room for a new entrant.  Let's examine another neighborhood, this time in the suburbs of Etobicoke

In [113]:
etobicoke_data = mergedDf[mergedDf['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
5,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
6,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
7,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
8,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [114]:
address = 'Etobicoke, CA'

geolocator = Nominatim(user_agent="etobicoke_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


### map of Etobicoke (Toronto)

In [115]:
# create map of Manhattan using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

In [116]:
etobicoke_data.loc[7, 'Neighbourhood']

'Alderwood, Long Branch'

In [117]:
neighbourhood_latitude = etobicoke_data.loc[7, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = etobicoke_data.loc[7, 'Longitude'] # neighborhood longitude value

neighbourhood_name = etobicoke_data.loc[7, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Alderwood, Long Branch are 43.60241370000001, -79.54348409999999.


In [118]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [119]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [120]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Il Paesano Pizzeria & Restaurant,Pizza Place,43.601280,-79.545028
1,Toronto Gymnastics International,Gym,43.599832,-79.542924
2,Timothy's Pub,Pub,43.600165,-79.544699
3,Pizza Pizza,Pizza Place,43.605340,-79.547252
4,Tim Hortons,Coffee Shop,43.602396,-79.545048
5,Subway,Sandwich Place,43.599152,-79.544395
6,Rexall,Pharmacy,43.601951,-79.545694


In [121]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


In [122]:
nearby_venues['categories']

0       Pizza Place
1               Gym
2               Pub
3       Pizza Place
4       Coffee Shop
5    Sandwich Place
6          Pharmacy
Name: categories, dtype: object

In [123]:
nearby_venues.groupby(by='categories').count()

,name,lat,lng
categories,,,
Coffee Shop,1,1,1
Gym,1,1,1
Pharmacy,1,1,1
Pizza Place,2,2,2
Pub,1,1,1
Sandwich Place,1,1,1


### There is only 1 coffee shop in this neighborhood; there are a couple of pizza places, and a sandwhich place.  This is a bettter neighborhood choice to open a coffee shop

### Here we will see a map of the venues available in this suburban location

In [124]:
# create map of Manhattan using latitude and longitude values
map_etobicoke_venues = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke_venues)
    
map_etobicoke_venues

### Yup - this looks much better.  Its not as dense as downtown, providing opportunity for a new entrant with only one other coffee shop nearby.